In [11]:
from google.cloud import bigquery
import pandas as pd

In [12]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/path/to/downloaded/google-credentials-json-file"

project_id = "propane-highway-326820"
client = bigquery.Client()

In [13]:
query = """
    SELECT * FROM `propane-highway-326820.twitter_stream.tweets`
"""
query_job = client.query(query)

In [14]:
output = pd.DataFrame()
for row in query_job:
    output = output.append(dict(row), ignore_index=True)
# print(output)

In [15]:
output.head()

,created_at,id,retweet_count,text,user_name,user_location,user_followers_count
0,Tue Oct 19 22:52:20 +0000 2021,1450595702065745920,0,@therecount Of course in the church of #covid ...,"JW, Domestic Terrorist","Panama City, FL",633
1,Tue Oct 19 22:52:27 +0000 2021,1450595729970454529,0,RT @American_Alley2: 👇Should never happen in A...,kung fu jedi,"Alabama, USA",14725
2,Tue Oct 19 22:53:56 +0000 2021,1450596105088114689,0,"RT @CDCgov: As of October 18, 2021, 219 millio...",ETSU Center for Rural Health Research,ETSU College of Public Health,164
3,Tue Oct 19 22:55:02 +0000 2021,1450596378313375750,0,RT @sdbaral: Just a FYI to anyone who cares th...,Panic in Needle Park,"Sarasota, FL",606
4,Tue Oct 19 22:55:14 +0000 2021,1450596432436736002,0,RT @BenPBradshaw: Very disappointing non-answe...,Steaks,Europe,225


In [16]:
#converting date from string to datetime
output['created_at'] = pd.to_datetime(output['created_at'])
# output


In [17]:
df = output['created_at'].groupby(output['created_at'].dt.floor('15T')).size().reset_index(name='count')
print (df)

                 created_at  count
0 2021-10-19 22:45:00+00:00    675
1 2021-10-19 23:00:00+00:00   1187
2 2021-10-19 23:15:00+00:00    581


In [18]:
import plotly.express as px

fig = px.line(df, x='created_at', y="count", labels={'created_at':'Time', 'count':'Count'}, title= 'Tweets per 15 mins')
fig.show()

In [19]:
full_grouped = output.groupby([ 'user_location']).size().reset_index(name='count')
full_grouped.sort_values(['count'])


,user_location,count
0,"Ca. Desert, USA",1
590,"Philadelphia, PA",1
591,Philippians 3:20 ✞,1
592,Philippines,1
593,"Phoenix, Arizona",1
...,...,...
404,"Los Angeles, CA",23
520,"New York, NY",30
72,Australia,32
510,"New Delhi, India.",34


In [20]:
fig = px.bar(full_grouped, x='user_location', y='count')
fig.show()